In [112]:
# TODO Is it de-anonymize or anonymize?

# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np

pn.extension()

%matplotlib inline

In [113]:
# Function definitions: TODO move to .py

## Data cleaning
def remove_strings_from_customer_names(original_customer_name):
    invalid_strings = ["PYMT", "DUE"]
    for invalid_item in invalid_strings:
        original_customer_name = original_customer_name.replace(invalid_item, "")
    return original_customer_name.strip()

def remove_numbers_from_customer_names(original_customer_name):
    remove_digits = str.maketrans('', '', digits) 
    return original_customer_name.translate(remove_digits).strip()  

def cleanup_customer_names(paying_customers_raw):
    paying_customers_cleanedup = []
    for customer in paying_customers_raw:
        corrected_customer_name = str(customer)
        #corrected_customer_name = str(customer).upper()
        corrected_customer_name = remove_strings_from_customer_names(corrected_customer_name)
        corrected_customer_name = remove_numbers_from_customer_names(corrected_customer_name)
        #corrected_customer_name = corrected_customer_name.title()
        paying_customers_cleanedup.append(corrected_customer_name)
    return paying_customers_cleanedup

def build_name_mapping(paying_customers_cleanedup):
    name_mapping = {}
    n = 1
    for customer in paying_customers_cleanedup:
        if not customer in name_mapping:
            name_mapping[customer] = "University " + str(n)
            n += 1
    return name_mapping

def read_name_mapping():
    with open(MAPPING_FILE_PATH, "r") as file:
        return json.loads(file.read())

def anonymize_customer_list(customer_list):
    anonymized_customer_list = []
    for customer in customer_list:
        anonymized_customer_list.append(customer_name_mapping[customer])
    return anonymized_customer_list


In [114]:
# Constants - TODO Move to .py
MAPPING_DIR                     = Path("../Resources/Mappings")
DATA_DIR_RAW                    = Path("../Resources/01_Raw")
DATA_DIR_ANONYMIZED             = Path("../Resources/02_Anonymized")
DATA_DIR_PREPROCESSED           = Path("../Resources/03_Preprocessed")
DATA_DIR_PROCESSED              = Path("../Resources/04_Processed")

MAPPING_FILE_PATH               = os.path.join(MAPPING_DIR, Path("CustomerNameMapping.json"))

RAW_ATLAS_FILE_PATH             = os.path.join(DATA_DIR_RAW, Path("ATLAS.csv"))
RAW_FORECAST_DATA_FILE_PATH     = os.path.join(DATA_DIR_RAW, Path("2021 forecast CSV.csv"))
RAW_REVENUE2020_FILE_PATH       = os.path.join(DATA_DIR_RAW, Path("Revenue2020.csv"))
RAW_REVENUE2020A_FILE_PATH      = os.path.join(DATA_DIR_RAW, Path("Revenue2020A.csv"))

ANON_ATLAS_FILE_PATH            = os.path.join(DATA_DIR_ANONYMIZED, Path("ATLAS.csv"))
ANON_FORECAST_DATA_FILE_PATH    = os.path.join(DATA_DIR_ANONYMIZED, Path("2021 forecast CSV.csv"))
ANON_REVENUE2020_FILE_PATH      = os.path.join(DATA_DIR_ANONYMIZED, Path("Revenue2020.csv"))
ANON_REVENUE2020A_FILE_PATH     = os.path.join(DATA_DIR_ANONYMIZED, Path("Revenue2020A.csv"))

In [115]:
def cleanup_date_string_list(date_string_list):
    date_list = []
    for date_string in date_string_list:
        try:
            date_list.append( pd.Timestamp(date_string.replace("//", "/"), tz="America/New_York") )
        except:
            print(f"Failed to parse: {date_string}")
    return date_list

def cleanup_dollar_string(dollars_string):
    return dollars_string.replace('$','').replace(',', '').astype('float')

def cleanup_dollar_string_list(dollars_list_in):
    dollars_list_out = []
    for dollars_string in dollars_list_in:
        try:
            dollars_list_out.append( cleanup_dollar_string(dollars_string) )
        except:
            print(f"Failed to parse: {dollars_string}")
    return dollars_list_out

In [116]:
# Read anonymized data
atlas           = pd.read_csv(ANON_ATLAS_FILE_PATH, index_col="Customers")
forecast        = pd.read_csv(ANON_FORECAST_DATA_FILE_PATH, index_col="Organization Name")
revenue2020     = pd.read_csv(ANON_REVENUE2020_FILE_PATH, index_col="Name")
revenue2020A    = pd.read_csv(ANON_REVENUE2020A_FILE_PATH, index_col="Payee Name")

In [117]:
# Clean up ATLAS data
atlas["Invoice Date"]       = cleanup_date_string_list(atlas["Invoice Date"])
atlas["Invoice Amount"]     = cleanup_dollar_string_list(atlas["Invoice Amount"])
atlas.head()

Failed to parse: $72,000.00
Failed to parse: $3,500.00
Failed to parse: $3,500.00
Failed to parse: $6,500.00
Failed to parse: $750.00
Failed to parse: $3,400.00
Failed to parse: $28,000.00
Failed to parse: $16,200.00
Failed to parse: $67,000.00
Failed to parse: $3,500.00
Failed to parse: $750.00
Failed to parse: $750.00
Failed to parse: $2,166.67
Failed to parse: $2,166.67
Failed to parse: $2,166.67
Failed to parse: $3,500.00
Failed to parse: $750.00
Failed to parse: $750.00
Failed to parse: $750.00
Failed to parse: $3,500.00
Failed to parse: $1,000.00
Failed to parse: $750.00
Failed to parse: $750.00
Failed to parse: $6,600.00
Failed to parse: $3,500.00
Failed to parse: $2,850.00
Failed to parse: $3,500.00
Failed to parse: $750.00
Failed to parse: $9,450.00
Failed to parse: $750.00
Failed to parse: $3,500.00
Failed to parse: $750.00
Failed to parse: $750.00
Failed to parse: $2,450.00
Failed to parse: $750.00
Failed to parse: $2,450.00
Failed to parse: $2,450.00
Failed to parse: $50,00

ValueError: Length of values (0) does not match length of index (669)

In [111]:
type(atlas["Invoice Date"].to_list()  )

list